# Overriding input data

This notebook runs an Energy Hub model for different time series data loaded from a seperate Excel file.
The model is run several times and the results are collected in a single `DataFrame` for analysis.

In [ ]:
import pandas as pd
from pyehub.energy_hub.ehub_model import EHubModel
from pyehub.outputter import print_section

Since none of the constraints are altered in this example, we only have a `main()` function running the original Energy Hub model.
+ Specify the input EXCEL file. _[Must be in current directory]._
+ Form the model.
+ We use a different file to specify the LOADS for the model. You can similarly override any existing data present in the original excel file.
+ We here run the model 8 times with different `Elec` and `Heat` loads specified in the `8760_loads.xlsx` file.

In [ ]:
excel_file = "test_file_all_constraints_work.xlsx"  # name of the excel file. [This must be in the current directory]
my_model = EHubModel(
    excel=excel_file
)  # instantiate our model. Nothing is solved at this point.

dataframe = pd.read_excel(
    "8760_loads.xlsx", dtype=float
)  # load yearly data from an Excel file
start_days = [
    0,
    50,
    100,
    150,
    200,
    250,
    300,
    350,
]  # solve the model for single days starting at these days
start_times = [start_day * 24 for start_day in start_days]

results = []
for start_time in start_times:
    # We enumerate the data so that's it in the range 0..23
    elec_data = dataframe["Elec"][start_time : start_time + 24]
    new_elec = {t: value for t, value in enumerate(elec_data)}
    my_model.LOADS[
        "Elec"
    ] = new_elec  # replace elec load data with the new one from the Excel file

    heat_data = dataframe["Heat"][start_time : start_time + 24]
    new_heat = {t: value for t, value in enumerate(heat_data)}
    my_model.LOADS[
        "Heat"
    ] = new_heat  # replace heat load data with the new one from the Excel file

    my_model.recompile()  # recompile the model to update the constraints to reflect the new loads
    results.append(my_model.solve())  # solve the model and add to the results dataframe

for i, result in enumerate(results):  # for each run
    solution = result["solution"]  # get the optimal solution

    print_section(
        f"Solution {i} starting at time t = {start_times[i]}",
        {  # print the header
            "total_cost": solution["total_cost"],  # print the total cost value
            "LOADS": solution["LOADS"],  # print the loads used
        },
    )